In [1]:
%matplotlib inline
%reset
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import scipy

import SchemDraw as schem # to draw schematics (install using 'pip install SchemDraw')
import SchemDraw.elements as e

import skrf as rf # scikit-rf (install using 'conda -c scikit-rf scikit-rf')

import sympy as s # sympy (install using 'conda -c sympy sympy')
from sympy import I # imaginary symbol
from sympy.simplify.fu import fu, TR7

rf.stylely() # enable scikit-rf plot styles
s.init_printing(use_latex='mathjax') # print sympy equations in MathJAX (for Jupyter)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
def mag2db(mag):
    return np.multiply(20.0, np.log10(mag))
def mag2pdb(mag):
    return np.multiply(10.0, np.log10(mag))
def db2mag(db):
    return np.power(10, np.divide(db, 20.0))
def pdb2mag(pdb):
    return np.power(10, np.divide(pdb, 10.0))
def rad2deg(rad):
    return np.multiply(np.divide(rad, (2*np.pi)), 360.0)
def deg2rad(deg):
    return np.multiply(np.divide(deg, 360.0), 2*np.pi)
def dbm2v(dbm, R = 50):
    # P = V^2 / R
    power = 1e-3 * np.power(10, np.divide(dbm, 10))
    return np.sqrt(power * (R*2))    
def v2dbm(v, R = 50):
    power = np.divide(np.power(v, 2), R*2)
    return 10 * np.log10(power/1e-3)

In [7]:
# Single tone harmonic distortion
A, w1, t = s.symbols('A \omega_1 t', real=True)
a1, a2, a3 = s.symbols('a_1 a_2 a_3')
Si = A * s.cos(w1*t)
display(Si)
So = a1*Si + a2*(Si**2) + a3*(Si**3)
So = s.expand(s.fu(So, measure=lambda x: -x.count_ops()))
display(So)
print(s.latex(So))

A⋅cos(\omega₁⋅t)

   3                      3                        2                        2 
3⋅A ⋅a₃⋅cos(\omega₁⋅t)   A ⋅a₃⋅cos(3⋅\omega₁⋅t)   A ⋅a₂⋅cos(2⋅\omega₁⋅t)   A ⋅
────────────────────── + ────────────────────── + ────────────────────── + ───
          4                        4                        2                2

                        
a₂                      
── + A⋅a₁⋅cos(\omega₁⋅t)
                        

\frac{3 a_{3}}{4} A^{3} \cos{\left (\omega_1 t \right )} + \frac{A^{3} a_{3}}{4} \cos{\left (3 \omega_1 t \right )} + \frac{A^{2} a_{2}}{2} \cos{\left (2 \omega_1 t \right )} + \frac{A^{2} a_{2}}{2} + A a_{1} \cos{\left (\omega_1 t \right )}


In [6]:
# Two tone harmonic distortion
A, w1, w2, t = s.symbols('A \omega_1 \omega_2 t', real=True)
a1, a2, a3 = s.symbols('a_1 a_2 a_3')
Si = A * s.cos(w1*t) + A * s.cos(w2*t)
display(Si)
So = a1*Si + a2*(Si**2) + a3*(Si**3)
So = s.expand(s.fu(So, measure=lambda x: -x.count_ops()))
display(So)
print(s.latex(Si))
print(s.latex(So))

A⋅cos(\omega₁⋅t) + A⋅cos(\omega₂⋅t)

   3                      3                          3                      3 
9⋅A ⋅a₃⋅cos(\omega₁⋅t)   A ⋅a₃⋅cos(3⋅\omega₁⋅t)   9⋅A ⋅a₃⋅cos(\omega₂⋅t)   A ⋅
────────────────────── + ────────────────────── + ────────────────────── + ───
          4                        4                        4                 

                         3                                      3             
a₃⋅cos(3⋅\omega₂⋅t)   3⋅A ⋅a₃⋅cos(\omega₁⋅t - 2⋅\omega₂⋅t)   3⋅A ⋅a₃⋅cos(\omeg
─────────────────── + ──────────────────────────────────── + ─────────────────
       4                               4                                      

                         3                                      3             
a₁⋅t + 2⋅\omega₂⋅t)   3⋅A ⋅a₃⋅cos(2⋅\omega₁⋅t - \omega₂⋅t)   3⋅A ⋅a₃⋅cos(2⋅\om
─────────────────── + ──────────────────────────────────── + ─────────────────
4                                      4                                      

                       2                        2

A \cos{\left (\omega_1 t \right )} + A \cos{\left (\omega_2 t \right )}
\frac{9 a_{3}}{4} A^{3} \cos{\left (\omega_1 t \right )} + \frac{A^{3} a_{3}}{4} \cos{\left (3 \omega_1 t \right )} + \frac{9 a_{3}}{4} A^{3} \cos{\left (\omega_2 t \right )} + \frac{A^{3} a_{3}}{4} \cos{\left (3 \omega_2 t \right )} + \frac{3 a_{3}}{4} A^{3} \cos{\left (\omega_1 t - 2 \omega_2 t \right )} + \frac{3 a_{3}}{4} A^{3} \cos{\left (\omega_1 t + 2 \omega_2 t \right )} + \frac{3 a_{3}}{4} A^{3} \cos{\left (2 \omega_1 t - \omega_2 t \right )} + \frac{3 a_{3}}{4} A^{3} \cos{\left (2 \omega_1 t + \omega_2 t \right )} + \frac{A^{2} a_{2}}{2} \cos{\left (2 \omega_1 t \right )} + \frac{A^{2} a_{2}}{2} \cos{\left (2 \omega_2 t \right )} + A^{2} a_{2} \cos{\left (\omega_1 t - \omega_2 t \right )} + A^{2} a_{2} \cos{\left (\omega_1 t + \omega_2 t \right )} + A^{2} a_{2} + A a_{1} \cos{\left (\omega_1 t \right )} + A a_{1} \cos{\left (\omega_2 t \right )}


In [12]:
# Problem 1a
10**(db2mag(-3) / 20.0) - 1

0.0849189356218

In [13]:
# Blocker apparent gain effect (w1 = blocker, w2 = desired, A = blocker amplitude, B = desired amplitude)
# Problem 1b
A, B, w1, w2, t = s.symbols('A B \omega_1 \omega_2 t', real=True)
a1, a2, a3 = s.symbols('a_1 a_2 a_3')
Si = A * s.cos(w1*t) + B * s.cos(w2*t)
display(Si)
So = a1*Si + a2*(Si**2) + a3*(Si**3)
So = s.expand(s.fu(So, measure=lambda x: -x.count_ops()))
display(So)
print(s.latex(Si))
print(s.latex(So))

A⋅cos(\omega₁⋅t) + B⋅cos(\omega₂⋅t)

   3                      3                          2                        
3⋅A ⋅a₃⋅cos(\omega₁⋅t)   A ⋅a₃⋅cos(3⋅\omega₁⋅t)   3⋅A ⋅B⋅a₃⋅cos(\omega₂⋅t)   3
────────────────────── + ────────────────────── + ──────────────────────── + ─
          4                        4                         2                

  2                                        2                                  
⋅A ⋅B⋅a₃⋅cos(2⋅\omega₁⋅t - \omega₂⋅t)   3⋅A ⋅B⋅a₃⋅cos(2⋅\omega₁⋅t + \omega₂⋅t)
───────────────────────────────────── + ──────────────────────────────────────
                 4                                        4                   

    2                        2           2                          2         
   A ⋅a₂⋅cos(2⋅\omega₁⋅t)   A ⋅a₂   3⋅A⋅B ⋅a₃⋅cos(\omega₁⋅t)   3⋅A⋅B ⋅a₃⋅cos(\
 + ────────────────────── + ───── + ──────────────────────── + ───────────────
             2                2                2                              

                               2                 

A \cos{\left (\omega_1 t \right )} + B \cos{\left (\omega_2 t \right )}
\frac{3 a_{3}}{4} A^{3} \cos{\left (\omega_1 t \right )} + \frac{A^{3} a_{3}}{4} \cos{\left (3 \omega_1 t \right )} + \frac{3 a_{3}}{2} A^{2} B \cos{\left (\omega_2 t \right )} + \frac{3 a_{3}}{4} A^{2} B \cos{\left (2 \omega_1 t - \omega_2 t \right )} + \frac{3 a_{3}}{4} A^{2} B \cos{\left (2 \omega_1 t + \omega_2 t \right )} + \frac{A^{2} a_{2}}{2} \cos{\left (2 \omega_1 t \right )} + \frac{A^{2} a_{2}}{2} + \frac{3 a_{3}}{2} A B^{2} \cos{\left (\omega_1 t \right )} + \frac{3 a_{3}}{4} A B^{2} \cos{\left (\omega_1 t - 2 \omega_2 t \right )} + \frac{3 a_{3}}{4} A B^{2} \cos{\left (\omega_1 t + 2 \omega_2 t \right )} + A B a_{2} \cos{\left (\omega_1 t - \omega_2 t \right )} + A B a_{2} \cos{\left (\omega_1 t + \omega_2 t \right )} + A a_{1} \cos{\left (\omega_1 t \right )} + \frac{3 a_{3}}{4} B^{3} \cos{\left (\omega_2 t \right )} + \frac{B^{3} a_{3}}{4} \cos{\left (3 \omega_2 t \right )} + \frac{B^{2} a_{2}}{2} \c

In [14]:
10**(db2mag(-2)/20.0)-1

0.0957624448258